In [1]:
import os
import cv2
import numpy as np
import joblib
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Paths configuration
chest_dir = r"D:\Materials\ML\img_classifier\chest_dir"     # chest X-ray folder
non_chest_dir = r"D:\Materials\ML\img_classifier\non_chest_dir"      # non-chest X-ray folder

# Images & labels arrays identefication
X = []
y = []

# HOG settings
hog_params = {
    "orientations": 9,
    "pixels_per_cell": (8, 8),
    "cells_per_block": (2, 2),
    "block_norm": "L2-Hys"
}

image_size = (128, 128)  # image size for each image

# Images preprocessing and features extraction
def process_images_from_folder(folder_path, label):
    for filename in os.listdir(folder_path):
        path = os.path.join(folder_path, filename)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            continue

        img = cv2.resize(img, image_size)
        features = hog(img, **hog_params)
        X.append(features)
        y.append(label)

# Apply the pre-processing function
process_images_from_folder(chest_dir, label=1)
process_images_from_folder(non_chest_dir, label=0)

# converting into numby
X = np.array(X)
y = np.array(y)

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Model evaluation
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


C:\Users\Adel_\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Accuracy: 0.9950142450142451
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       675
           1       1.00      0.99      1.00       729

    accuracy                           1.00      1404
   macro avg       0.99      1.00      1.00      1404
weighted avg       1.00      1.00      1.00      1404



In [3]:
# Model inference
# Pre-processing pipeline
def predict_image(image_path):
    hog_params = {
    "orientations": 9,
    "pixels_per_cell": (8, 8),
    "cells_per_block": (2, 2),
    "block_norm": "L2-Hys"
    }
    image_size = (128, 128)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, image_size)
    features = hog(img, **hog_params)
    prediction = clf.predict([features])[0]
    probability = clf.predict_proba([features])[0]
    print(f"Prediction: {'Chest X-ray' if prediction == 1 else 'Other'}") # Print the prediction
    print(f"Probability: {probability}") # Print probabilities

# Inference on unseen image (Test)
predict_image(r"C:\Users\Adel_\OneDrive\Desktop\pneumonia.jpg")

Prediction: Chest X-ray
Probability: [3.94866642e-08 9.99999961e-01]


In [4]:
# save the best model
joblib.dump(clf, "img_classifier.pkl")

#load the model
clf = joblib.load(r"D:\Materials\ML\img_classifier\img_classifier.pkl")

In [6]:
# Loaded model inference
predict_image(r"C:\Users\Adel_\OneDrive\Desktop\pneumonia1.png")

Prediction: Chest X-ray
Probability: [0.0071638 0.9928362]
